In [1]:
import os 
import sys 
parent_dir = os.path.dirname(os.getcwd())
sys.path.append("/home/jbc0071/Documents/CANIDS/code")
# sys.path.append(parent_dir + "/code")

from model.CANnoloAttackDetector import CANnoloAttackDetector
from data_helpers.CANDataset import CANDataset
from helpers import calculate_feature_vec_length, seperate_attack_loader, calculate_metrics
from dotenv import load_dotenv
# from data_helpers.CANDataLoader import CANDataLoader
import os

load_dotenv()
data_path = os.getenv('DATA_PATH')
dataset = CANDataset(data_path, log_verbosity=1)
print()

config = {
    "batch_size": 32,
    "delta_time_last_msg": {
        "specific_to_can_id": False,
        "records_back": 30
    },
    "delta_time_last_same_aid": {
        "specific_to_can_id": True,
        "records_back": 15
    },
}


ambient_loader, validation_loader, attack_loader = dataset.get_dataloaders(config)

attack_loaders = seperate_attack_loader(attack_loader, config, remove_non_labelled=True)

Found ambient and attack directories.
Loading CAN metadata...
Parquet files found...
Found processed parquet files...
Loading processed parquet files...
Loading processing data into 'CANData' structure

No attack labels in accelerator_attack_drive_1.log
No attack labels in accelerator_attack_drive_2.log
No attack labels in accelerator_attack_reverse_1.log
No attack labels in accelerator_attack_reverse_2.log
Found attack labels in correlated_signal_attack_1.log
Found attack labels in correlated_signal_attack_2.log
Found attack labels in correlated_signal_attack_3.log
Found attack labels in fuzzing_attack_1.log
Found attack labels in fuzzing_attack_2.log
Found attack labels in fuzzing_attack_3.log
Found attack labels in max_engine_coolant_temp_attack.log
Found attack labels in max_speedometer_attack_1.log
Found attack labels in max_speedometer_attack_2.log
Found attack labels in max_speedometer_attack_3.log
Found attack labels in reverse_light_off_attack_1.log
Found attack labels in reve

In [2]:

unique_can_ids = dataset.get_unique_can_ids()
num_can_ids = len(unique_can_ids)
feature_vec_length = ambient_loader.features_len

# Load model
model_path = 'models/canolo_model_112.pt'

model_config = {
    "embedding_dim": num_can_ids,
    "lstm_units": 128,
    "dense_units": 256,
    "dropout_rate": 0.2,
    "num_embeddings": max(unique_can_ids) + 1, # not sure why + 1 rn but it works
    "feature_vec_length": calculate_feature_vec_length(config)
}

threshold = 5.9000547253162944e-06

In [4]:
for loader in attack_loaders:
    loader.reset()

In [5]:
from concurrent.futures import ProcessPoolExecutor
from detect import detect_wrapper
import time

num_attack_files = len(attack_loaders)
detectors = []

start_time = time.time()
for _ in range(num_attack_files):
    detector = CANnoloAttackDetector(model_path, threshold, model_config, force_cpu=True)
    detectors.append(detector)

with ProcessPoolExecutor() as executor:
    list_of_filename_and_results = list(executor.map(detect_wrapper, zip(attack_loaders, detectors)))

for loader in attack_loaders:
    loader.reset()
end_time = time.time()

print(f"Finished in {end_time - start_time} seconds")

print(list_of_filename_and_results)

Forcing CPU usage...
Forcing CPU usage...
Forcing CPU usage...
Forcing CPU usage...
Forcing CPU usage...
Forcing CPU usage...
Forcing CPU usage...
Forcing CPU usage...
Forcing CPU usage...
Forcing CPU usage...
Forcing CPU usage...
Forcing CPU usage...
Forcing CPU usage...
Forcing CPU usage...
Forcing CPU usage...
Forcing CPU usage...
Testing on correlated_signal_attack_1.log
Testing on correlated_signal_attack_2.log
Testing on correlated_signal_attack_3.log
Testing on fuzzing_attack_1.log
Testing on fuzzing_attack_2.log
Testing on fuzzing_attack_3.log
Testing on max_engine_coolant_temp_attack.log
Testing on max_speedometer_attack_1.log
Testing on max_speedometer_attack_2.log
Testing on reverse_light_off_attack_1.logTesting on max_speedometer_attack_3.log

Testing on reverse_light_off_attack_2.log
Testing on reverse_light_off_attack_3.log
Testing on reverse_light_on_attack_1.log
Testing on reverse_light_on_attack_3.log
Testing on reverse_light_on_attack_2.log
Finished in 174.11441898345

In [6]:
[('correlated_signal_attack_1.log', (0.9418705375892482, 0.0, 0.0, 0, [[0, 2343], [2086, 71763]])), ('correlated_signal_attack_2.log', (0.9291136141038198, 0.31590607000443066, 1.0, 0.48013468013468014, [[2139, 4632], [0, 58573]])), ('correlated_signal_attack_3.log', (0.9306122448979591, 0.31709766507657544, 1.0, 0.4815097216927182, [[1263, 2720], [0, 35217]])), ('fuzzing_attack_1.log', (0.9804385964912281, 0.002325581395348837, 0.05555555555555555, 0.004464285714285714, [[2, 858], [34, 44706]])), ('fuzzing_attack_2.log', (0.9635810810810811, 0.002806361085126286, 0.2, 0.005535055350553506, [[3, 1066], [12, 28519]])), ('fuzzing_attack_3.log', (0.95, 0.001669449081803005, 0.3333333333333333, 0.003322259136212625, [[1, 598], [2, 11399]])), ('max_engine_coolant_temp_attack.log', (0.9706988134657837, 0.0, 0.0, 0, [[0, 1657], [42, 56285]])), ('max_speedometer_attack_1.log', (0.9696447136916186, 0.0, 0.0, 0, [[0, 3629], [2444, 193991]])), ('max_speedometer_attack_2.log', (0.9591615996258185, 0.0, 0.0, 0, [[0, 2448], [3140, 131244]])), ('max_speedometer_attack_3.log', (0.8959264729362925, 0.0, 0.0, 0, [[0, 14751], [6107, 179558]])), ('reverse_light_off_attack_1.log', (0.9643171529175051, 0.0, 0.0, 0, [[0, 1598], [672, 61346]])), ('reverse_light_off_attack_2.log', (0.9560145547945206, 0.0005743825387708214, 0.00042176296921130323, 0.0004863813229571984, [[1, 1740], [2370, 89329]])), ('reverse_light_off_attack_3.log', (0.9632148049430579, 0.0, 0.0, 0, [[0, 2424], [2434, 127206]])), ('reverse_light_on_attack_1.log', (0.9524884704073789, 0.0, 0.0, 0, [[0, 3943], [1991, 118962]])), ('reverse_light_on_attack_2.log', (0.9440929804149699, 0.0, 0.0, 0, [[0, 5537], [3689, 155798]])), ('reverse_light_on_attack_3.log', (0.9539725503062118, 0.0, 0.0, 0, [[0, 4383], [2351, 139570]]))]

[('correlated_signal_attack_1.log',
  (0.9418705375892482, 0.0, 0.0, 0, [[0, 2343], [2086, 71763]])),
 ('correlated_signal_attack_2.log',
  (0.9291136141038198,
   0.31590607000443066,
   1.0,
   0.48013468013468014,
   [[2139, 4632], [0, 58573]])),
 ('correlated_signal_attack_3.log',
  (0.9306122448979591,
   0.31709766507657544,
   1.0,
   0.4815097216927182,
   [[1263, 2720], [0, 35217]])),
 ('fuzzing_attack_1.log',
  (0.9804385964912281,
   0.002325581395348837,
   0.05555555555555555,
   0.004464285714285714,
   [[2, 858], [34, 44706]])),
 ('fuzzing_attack_2.log',
  (0.9635810810810811,
   0.002806361085126286,
   0.2,
   0.005535055350553506,
   [[3, 1066], [12, 28519]])),
 ('fuzzing_attack_3.log',
  (0.95,
   0.001669449081803005,
   0.3333333333333333,
   0.003322259136212625,
   [[1, 598], [2, 11399]])),
 ('max_engine_coolant_temp_attack.log',
  (0.9706988134657837, 0.0, 0.0, 0, [[0, 1657], [42, 56285]])),
 ('max_speedometer_attack_1.log',
  (0.9696447136916186, 0.0, 0.0, 0, [